# 任务

编译原理实验内容：

实验一、扫描器的设计与实现 词法分析任务是从以字符串表示的源程序中识别出具有独立意义的基本语法单位（或语法符号），同时指出它们的属性。 
1、要求程序能识别出基本字，标识符，常数，运算符和界符，并且有适当的报错功能，基本字集可选取C语言基本字集的一个子集。 
2、使用文件操作， 输入文件：C语言源程序 输出文件：二元式序列（亦称TOKEN序列） 

下列三个实验完成一个即可 
- 实验二、算符优先分析设计与实现（P79 图4.6） 
- 实验三、LL(1)方法即预测分析方法设计与实现 
- 实验四、递归下降分析程序设计与实现 实验可参考附录C，

建议使用C/VC语言调试程序；也可以使用Python语言等 ，

实验一是必须要写的，实验2，3，4任选一个。


# Python 写个 C 子集的 Parser

目标是从文件读入源程序，输出 token 序列

## 设计

文件读入，转成字符流，记录 pos

parse 基本字，标识符，常数，运算符和界符

```

if for else return

字母+数字+_

数字

+ - / *

[] {} () " ; ,

```

面向对象吧...

In [2]:
import sys,os

class CStream:
    sourceFile = ''
    index = 0
    line = 1
    pos = 0
    
    def __init__(self,path):
        with open(path, 'r') as f:
            self.sourceFile = f.read()
            
    def checkNext(self):
        return self.sourceFile[self.index]
            
    def getNext(self):
        ret = self.sourceFile[self.index]
        if ret == '\r':
            self.pos = 0
            self.line += 1
        else:
            self.pos += 1
        self.index += 1
        return ret
    
    def hasNext(self):
        return self.index < len(self.sourceFile)
    
    def err(self):
        print("parse error! at line %d , pos %d" %(self.line,self.pos))
        
    def printFile(self):
        print(self.sourceFile)
    

class Parser:
    word = ''
    baseWord = ('if','else','for','int','void','return')
    optWord = ('+','-','*','/','==','=')
    boundWord = ('[',']','{','}','(',')',',',';')
    ret = ''
    
    def __init__(self,cstream):
        self.stream = cstream
        
    def cleanWord(self):
        self.word = ''
        
    def parse(self):
        while(self.stream.hasNext()):
            ch = self.stream.checkNext()
            if ch in self.optWord:
                self.parseOpt()
            elif ch in self.boundWord:
                self.ret += '%s, boundary;\n' %self.stream.getNext()
            elif ch.isdigit():
                self.parseNum()
            elif ch.isalpha() or ch == '_':
                self.parseWordOrBaseWord()
            elif ch == ' ' or ch =='\n':
                self.stream.getNext()
                continue
            else:
                self.stream.err()
                return
        print(self.ret)
        with open('output.token','w+') as f:
            f.write(self.ret)
        
    def parseOpt(self):
        self.word += self.stream.getNext()
        if self.word == '=' and self.stream.checkNext() == '=':
            self.word += self.stream.getNext()
        self.ret += '%s, opt;\n' %self.word
        self.cleanWord()
    
    def parseWordOrBaseWord(self):
        self.scanWord()
        if self.word in self.baseWord:
            self.ret += '%s, baseWord;\n' %self.word
        else:
            self.ret += '%s, word;\n' %self.word
        self.cleanWord()
    
    def parseNum(self):
        self.scanWord()
        for i in range(0,len(self.word)):
            if not self.word[i].isdigit():
                self.stream.err()
        self.ret += '%s, num;\n' %self.word
        self.cleanWord()
        
    def scanWord(self):
        while(self.stream.hasNext()):
            ch = self.stream.checkNext()
            if not (ch.isalpha() or ch.isdigit() or ch == '_'):
                break
            self.word += self.stream.getNext()
        

s = CStream('target.c')
ps = Parser(s)

print("====source====")
s.printFile()
print("====token====")
ps.parse()

====source====
int main(){
    int a = 1;
    int b = 2;
    add(a,b);
    return 0;
}
====token====
int, baseWord;
main, word;
(, boundary;
), boundary;
{, boundary;
int, baseWord;
a, word;
=, opt;
1, num;
;, boundary;
int, baseWord;
b, word;
=, opt;
2, num;
;, boundary;
add, word;
(, boundary;
a, word;
,, boundary;
b, word;
), boundary;
;, boundary;
return, baseWord;
0, num;
;, boundary;
}, boundary;

